## import packages and read data

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import warnings
warnings.filterwarnings('ignore')

## read data and EDA

In [2]:
path = os.path.join(os.getcwd(), 'delivery_orders_march.csv')
df = pd.read_csv(path)
df.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


## retrieve the state from the address


In [3]:
df['buyeraddress_state'] = df['buyeraddress'].apply(lambda x: x.rsplit(maxsplit=1)[-1].lower())
df['selleraddress_state'] = df['selleraddress'].apply(lambda x: x.rsplit(maxsplit=1)[-1].lower())
df
# df['buyeraddress_state'].isnull().values.any()
# df['selleraddress_state'].isnull().values.any()
# abc.isnull().sum()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress,buyeraddress_state,selleraddress_state
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...,manila,manila
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ...",manila,manila
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M...",manila,manila
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number...",manila,manila
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...,luzon,manila
...,...,...,...,...,...,...,...,...
3176308,31504087640510,1585821728,1.585978e+09,NaN,"mayapis. edeliza JC274 5542 brgy.Dita, Metro M...",Intimate St) compound. Sekiat 2B8 98-B Mel far...,manila,manila
3176309,31504147352227,1585895011,1.586179e+09,1.586232e+09,"Superdome PORKY K307 Libertad, PCGCC sub.bagui...","13th 3616 Pilipinas, :1v26 1f-5 (Village) bl10...",manila,manila
3176310,31504462290482,1585826951,1.585965e+09,NaN,Galay's 1republic 333-18 Diliman 74d Malbay fe...,"Lane, 379B 1739A Ivc AFPFC Cainta, Waterpark, ...",manila,manila
3176311,31504851495943,1585891194,1.586230e+09,NaN,"#4560, 80south biloca vi!!age KM21, MANRESA, ,...",lrt-2 Consolacion 224-A Paradise 6D 36st champ...,luzon,manila


## delivery standards: SLA(Service Level Agreements ) matrix

1. Working Days are defined as "Mon - Sat", excluding Public Holidays.
2. SLA calculation begins from the next day after pickup (Day 0 = Day of Pickup; Day 1 = Next Day after Pickup)
3. 2nd Attempt must be no later than 3 working days after the 1st Attempt, regardless of origin to destination route (Day 0 = Day of 1st Attempt; Day 1 = Next Day after 1st Attempt).

In [4]:
origin_seller=['Metro Manila','Luzon','Visayas','Mindanao']
destination_buyer=['Metro Manila','Luzon','Visayas','Mindanao']
working_days = [[3,5,7,7],[5,5,7,7],[7,7,7,7],[7,7,7,7]]
sla_matrix = pd.DataFrame(working_days,index=origin_seller,columns=destination_buyer)
sla_matrix.style.set_caption("1st Attempt SLA (Working Days)")

,Metro Manila,Luzon,Visayas,Mindanao
Metro Manila,3,5,7,7
Luzon,5,5,7,7
Visayas,7,7,7,7
Mindanao,7,7,7,7


## map standard delivery days to each order 

In [5]:
res = []

for row in df.itertuples(index=False):
    
    ori = row.selleraddress_state
    des = row.buyeraddress_state
    
    if ori == 'manila' and des == 'manila':
        res.append(3)
    elif (ori == 'manila' and des == 'luzon') or (ori == 'luzon' and des == 'manila') or (ori == 'luzon' and des == 'luzon'):
        res.append(5)
    else:
        res.append(7)

df['delivery_days'] = res       

## classify whether the orders were reshipped

In [6]:
# orders that were not reshipped： 2nd_deliver_attempt == NaN
nan = df[df['2nd_deliver_attempt'].isna()].reset_index(drop = True)

In [7]:
# orders that were reshipped： 2nd_deliver_attempt != NaN
notna = df[df['2nd_deliver_attempt'].notna()].reset_index(drop = True)

## convert timestamps to dates and store into new columns

In [8]:
# epoch time to GMT+8
nan['pick_date'] = pd.to_datetime(nan['pick'], unit='s') + datetime.timedelta(hours = 8)
nan['pick_date'] = nan['pick_date'].dt.date

nan['1st_date'] = pd.to_datetime(nan['1st_deliver_attempt'], unit='s') + datetime.timedelta(hours = 8)
nan['1st_date'] = nan['1st_date'].dt.date

nan = nan.drop(columns=['pick','1st_deliver_attempt','2nd_deliver_attempt','buyeraddress','selleraddress'])
nan

,orderid,buyeraddress_state,selleraddress_state,delivery_days,pick_date,1st_date
0,2215676524,manila,manila,3,2020-03-02,2020-03-05
1,2220979489,manila,manila,3,2020-03-04,2020-03-06
2,2221066352,manila,manila,3,2020-03-05,2020-03-07
3,2222478803,luzon,manila,5,2020-03-04,2020-03-06
4,2222597288,manila,manila,3,2020-03-04,2020-03-07
...,...,...,...,...,...,...
1819306,31502339045455,manila,manila,3,2020-04-03,2020-04-07
1819307,31503788156046,manila,manila,3,2020-04-03,2020-04-04
1819308,31504087640510,manila,manila,3,2020-04-02,2020-04-04
1819309,31504462290482,manila,manila,3,2020-04-02,2020-04-04


In [9]:
notna['pick_date'] = pd.to_datetime(notna['pick'], unit='s') + datetime.timedelta(hours = 8)
notna['pick_date'] = notna['pick_date'].dt.date

notna['1st_date'] = pd.to_datetime(notna['1st_deliver_attempt'], unit='s') + datetime.timedelta(hours = 8)
notna['1st_date'] = notna['1st_date'].dt.date

notna['2nd_date'] = pd.to_datetime(notna['2nd_deliver_attempt'], unit='s') + datetime.timedelta(hours = 8)
notna['2nd_date'] = notna['2nd_date'].dt.date

notna = notna.drop(columns=['pick','1st_deliver_attempt','2nd_deliver_attempt','buyeraddress','selleraddress'])
notna

,orderid,buyeraddress_state,selleraddress_state,delivery_days,pick_date,1st_date,2nd_date
0,2219624609,manila,manila,3,2020-03-04,2020-03-06,2020-03-10
1,2222738456,manila,manila,3,2020-03-02,2020-03-05,2020-03-09
2,2224704587,luzon,manila,5,2020-03-04,2020-03-05,2020-03-09
3,2227771109,manila,manila,3,2020-03-02,2020-03-04,2020-03-05
4,2227783105,manila,manila,3,2020-03-02,2020-03-05,2020-03-07
...,...,...,...,...,...,...,...
1356997,31497388770741,manila,manila,3,2020-04-03,2020-04-06,2020-04-07
1356998,31500223560475,manila,manila,3,2020-04-03,2020-04-06,2020-04-07
1356999,31502490611412,luzon,manila,5,2020-04-02,2020-04-04,2020-04-06
1357000,31504147352227,manila,manila,3,2020-04-03,2020-04-06,2020-04-07


## convert date to integer

In [10]:
nan['pick_transform'] = (nan['pick_date']- datetime.date(2020,3,1)).dt.days + 1
nan['1st_transform']  = (nan['1st_date']- datetime.date(2020,3,1)).dt.days + 1

notna['pick_transform'] = (notna['pick_date']- datetime.date(2020,3,1)).dt.days + 1
notna['1st_transform']  = (notna['1st_date']- datetime.date(2020,3,1)).dt.days + 1
notna['2nd_transform']  = (notna['2nd_date']- datetime.date(2020,3,1)).dt.days + 1

## check if shipment is delayed

In [11]:
not_working_days = [1,8,15,22,25,29,30,31,36]

In [12]:
nan['working_days'] = nan['1st_transform'] - nan['pick_transform']

for ii in not_working_days:
    nan.loc[(nan['pick_transform'] < ii) & (nan['1st_transform'] > ii), 'working_days'] -= 1

In [14]:
nan['is_late'] = np.where(nan['working_days'] > nan['delivery_days'],1,0)
nan

,orderid,buyeraddress_state,selleraddress_state,delivery_days,pick_date,1st_date,pick_transform,1st_transform,working_days,is_late
0,2215676524,manila,manila,3,2020-03-02,2020-03-05,2,5,3,0
1,2220979489,manila,manila,3,2020-03-04,2020-03-06,4,6,2,0
2,2221066352,manila,manila,3,2020-03-05,2020-03-07,5,7,2,0
3,2222478803,luzon,manila,5,2020-03-04,2020-03-06,4,6,2,0
4,2222597288,manila,manila,3,2020-03-04,2020-03-07,4,7,3,0
...,...,...,...,...,...,...,...,...,...,...
1819306,31502339045455,manila,manila,3,2020-04-03,2020-04-07,34,38,3,0
1819307,31503788156046,manila,manila,3,2020-04-03,2020-04-04,34,35,1,0
1819308,31504087640510,manila,manila,3,2020-04-02,2020-04-04,33,35,2,0
1819309,31504462290482,manila,manila,3,2020-04-02,2020-04-04,33,35,2,0


In [15]:
notna['working_days_1'] = notna['1st_transform'] - notna['pick_transform']
notna['working_days_2'] = notna['2nd_transform'] - notna['1st_transform']

for ii in not_working_days:
    notna.loc[(notna['pick_transform'] < ii) & (notna['1st_transform'] > ii), 'working_days_1'] -= 1
    notna.loc[(notna['1st_transform'] < ii) & (notna['2nd_transform'] > ii), 'working_days_2'] -= 1

In [16]:
notna['is_late'] = np.where(notna['working_days_1'] > notna['delivery_days'],1,0)
is_late_2 = np.where(notna['working_days_2'] > 3,1,0)
notna['is_late'].loc[is_late_2 == 1] = 1
notna

,orderid,buyeraddress_state,selleraddress_state,delivery_days,pick_date,1st_date,2nd_date,pick_transform,1st_transform,2nd_transform,working_days_1,working_days_2,is_late
0,2219624609,manila,manila,3,2020-03-04,2020-03-06,2020-03-10,4,6,10,2,3,0
1,2222738456,manila,manila,3,2020-03-02,2020-03-05,2020-03-09,2,5,9,3,3,0
2,2224704587,luzon,manila,5,2020-03-04,2020-03-05,2020-03-09,4,5,9,1,3,0
3,2227771109,manila,manila,3,2020-03-02,2020-03-04,2020-03-05,2,4,5,2,1,0
4,2227783105,manila,manila,3,2020-03-02,2020-03-05,2020-03-07,2,5,7,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356997,31497388770741,manila,manila,3,2020-04-03,2020-04-06,2020-04-07,34,37,38,2,1,0
1356998,31500223560475,manila,manila,3,2020-04-03,2020-04-06,2020-04-07,34,37,38,2,1,0
1356999,31502490611412,luzon,manila,5,2020-04-02,2020-04-04,2020-04-06,33,35,37,2,1,0
1357000,31504147352227,manila,manila,3,2020-04-03,2020-04-06,2020-04-07,34,37,38,2,1,0


## output

In [20]:
col = ['orderid', 'is_late']
result = pd.concat([nan[col], notna[col]], ignore_index=True)
result

,orderid,is_late
0,2215676524,0
1,2220979489,0
2,2221066352,0
3,2222478803,0
4,2222597288,0
...,...,...
3176308,31497388770741,0
3176309,31500223560475,0
3176310,31502490611412,0
3176311,31504147352227,0


In [ ]:
result.to_csv('submission_20200711_data_analytics.csv',index=False)